<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/Text%20to%20SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 설정

## 1.1. 일반설정

- 모든 답변, 생성하는 마크다운이나 코드의 주석은 모두 한국어로 생성할것. 코드 외에 영어 사용은 자제.

## 1.2. Bigquery 설정

- 클라우드 빅쿼리에 project_id = 'river-nectar-469806-u4' dataset_id = 'loan' table_id => 'tb_a', 'tb_b', 'tb_c', 'tb_d' 이 정보를 바탕으로 쿼리 생성할 준비 마쳐주세요. 계정연결이 필요한 경우 진행해주세요.
- 빅쿼리 쿼리 작성시 별칭으로 한글 별칭은 삼가주세요. 다만 데이터 추출 후 파이썬 판다스로 데이터테이블 작업시에는 한글컬럼, 한글인덱스명을 사용해주세요.

## 1.3. python 설정

- 빅쿼리 결과물은 항상 데이터프레임에 저장하도록 할 것.
- 그래프는 항상 plotly_express를 사용하여 생성할 것.
- 파이썬 코드 실행 시 발생하는 모든 경고 메시지를 무시
- 테이블, 그래프, 대시보드 출력시에는 매번 출력창 높이 제한을 해제하기 위해 다음의 코드를 포함시켜 실행할 것.
  ```
  from IPython.display import Javascript, display
  display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: Infinity})'''))
  ```

## 1.4. 대시보드 설정

- 사용자가 분석을 정리하는 차원에서 대시보드를 만드려고 하면 반드시 jupyter-dash 라이브러리를 사용하여 대시보드를 만들것.
- jupyter-dash, dash-table, dash 라이브러리를 설치할것.
- from dash import dcc와 from dash import html로 임포트할것
- 사용자가 대시보드에 넣기 원하는 그래프, 테이블(판다스 데이터프레임인 경우 dash테이블로 전환하여 넣도록 할 것), 서술내용이 들어간 마크다운을 적절히 배치하여 완성도 높은 대시보드를 만들것.
- 사용자가 많은 내용을 넣으려고 하면 SyntaxError가 발생할 가능성이 높음. 이는 Dash 컴포넌트들을 중첩하여 배치하고 리스트 컴프리헨션을 사용하는 과정에서 발생한 파싱 문제일 가능성이 높음. 복잡한 레이아웃 구조를 정의할 때 Dash 컴포넌트를 올바르게 구성하도록 각별히 주의할것.
- 출력모드는 선택할수 있음. 코랩내부에서 보고싶다면 inline으로, 별도의 탭으로 보고 싶다면 external로
  - inline모드
    ```
    # 'inline' 모드로 실행하되, 출력창 높이를 1600px로 설정
    app.run(jupyter_mode='inline', jupyter_height=1600)
    ```
  - external모드
    ```
    # 'external' 모드로 실행.
    app.run(jupyter_mode='external')
    ```

# 2. 테이블 정보

## 2.1. 테이블 컬럼 명세

### 2.1.1. tb_a— 신청 마스터 (Application Master)

| 컬럼명 | 한글명 | 타입 (BigQuery) | 값/도메인 | 비고 |
|---|---|---|---|---|
| `loan_req_id` | 신청번호 | STRING | `R000001` ~ `R300000` | **PK** |
| `customer_id` | 고객번호 | STRING | `C000001` ~ `C300000` |  |
| `gender` | 성별 | STRING | `남`, `여` |  |
| `age` | 나이 | INT64 | 20 ~ 65 | 정수 |
| `income` | 소득 | INT64 | 0 ~ 999,999,999 | 원 단위 정수 |
| `realestate` | 거주형태 | STRING | `월세`, `전세`, `자가`, `가족과거주` |  |
| `occupation_cd` | 직업구분 | STRING | `자영업자`, `급여소득자`, `주부` |  |
| `application_date` | 신청일자 | DATE | 2025-04-01 ~ 2025-06-30 |  |
| `process_date` | 처리일자 | DATE | 2025-04-01 ~ 2025-06-30 |  |
| `inquiry_id` | 선택조회ID | STRING | `B000001`… 또는 NULL | tb_b 참조 |
| `final_loan_amount` | 최종대출금액 | FLOAT64 | NULL 또는 ≥ 0 | 실행 시 채움 |
| `final_rate` | 최종금리 | FLOAT64 | NULL 또는 > 0 | 실행 시 채움 |
| `execution_date` | 대출실행일자 | DATE | 2025-04-01 ~ 2025-06-30 |  |

### 2.1.2. tb_c— 한도/금리 조회 이력 (Limit Inquiry & Offer)

| 컬럼명 | 한글명 | 타입 (BigQuery) | 값/도메인 | 비고 |
|---|---|---|---|---|
| `loan_req_id` | 신청번호 | STRING | `R000001`… | tb_a 참조 |
| `inquiry_id` | 한도조회ID | STRING | `B000001`… | 재조회로 다건 가능 |
| `table_reg_no` | 등록순번 | INT64 | 1…(증가) | **유일키** |
| `product_id` | 상품구분 | STRING | `신용`, `오토` |  |
| `cb_score` | CB점수 | INT64 | 0 ~ 999 |  |
| `as_grade` | AS등급 | INT64 | 1 ~ 10 |  |
| `int_grade` | 금리등급 | INT64 | 1 ~ 6 |  |
| `limit_grade` | 한도등급 | INT64 | 1 ~ 6 |  |
| `inquiry_date` | 조회일자 | DATE | 2025-04-01 ~ 2025-06-30 |  |
| `offer_limit` | 제안한도 | INT64 | 0 또는 양의 정수 | 0 = 부여 없음 |
| `offer_rate` | 제안금리 | FLOAT64 | 0 또는 양의 실수 | 0 = 부여 없음 |
| `channel` | 채널 | STRING | `Toss`, `Kakao`, `Naver Pay`, `Direct`, `In-Direct` |  |
| `seg` | 세그먼트 | STRING | `SEG1`, `SEG2`, `SEG3`, `SEG4`, `SEG5` |  |

### 2.1.3. tb_c — 접수 상세 (Reception Detail)

| 컬럼명 | 한글명 | 타입 (BigQuery) | 값/도메인 | 비고 |
|---|---|---|---|---|
| `loan_req_id` | 신청번호 | STRING | `R000001`… | **PK**, tb_a 참조 |
| `reception_yn` | 접수여부 | BOOLEAN | true, false, NULL |  |
| `reception_product_id` | 접수상품 | STRING | `신용`, `오토` | tb_b의 `product_id`와 연계 |
| `reception_channel` | 접수채널 | STRING | `Toss`, `Kakao`, `Naver Pay`, `Direct`, `In-Direct` |  |

​

## 2.2. 테이블 로직 명세

테이블 로직 명세 (신청→조회→접수→실행)

- 본 데이터는 단순 관계가 아니라 실제 업무 흐름을 반영한다. 쿼리/집계는 반드시 이 순서를 전제.


테이블/관계

* tb_a(신청): PK=loan_req_id. 주요: application_date, process_date, inquiry_id, final_loan_amount, final_rate, execution_date.

* tb_b(한도조회): 유일키=table_reg_no. 동일 inquiry_id 재조회 다수. 주요: loan_req_id, inquiry_id, product_id, channel, inquiry_date, offer_limit, offer_rate.

* tb_c(접수): PK=loan_req_id. 주요: reception_yn(true/false/NULL), reception_product_id, reception_channel.

* 관계: A:B=1:N(loan_req_id), A:C=1:1. 최종 선택은 A.inquiry_id = B.inquiry_id. C↔B 연결은 C.loan_req_id=B.loan_req_id AND C.reception_product_id=B.product_id.


단계 요약

1. 신청: A에 1건 생성(상품/한도 미확정).

2. 한도조회: 상품별 제안 다수. B_latest(inquiry_id별 최신 1행) 생성: 정렬 table_reg_no DESC(BQ: ROW_NUMBER… QUALIFY rn=1).

3. 접수: 선택 시 C 기록, 미접수도 C에 기록(reception_yn='N').

* 집계 축 권장: kpi_channel = COALESCE(C.reception_channel, B_latest.channel), kpi_product = B_latest.product_id.

4. 실행: 접수된 건만 실행 기록(final_loan_amount, execution_date). 실행금액 ≤ 선택 제안 offer_limit. 미접수는 실행 NULL.


권장 조인

* A↔B: A.loan_req_id=B.loan_req_id(1:N), 선택상품은 A.inquiry_id=B.inquiry_id

* A↔C: A.loan_req_id=C.loan_req_id(1:1)

* C↔B: C.loan_req_id=B.loan_req_id AND C.reception_product_id=B.product_id


퍼널/지표 정의

* applications = COUNT(A)

* with_limit = (B_latest.offer_limit > 0)

* receptions = (C.reception_yn=true)

* executions = (A.final_loan_amount IS NOT NULL)


무결성/검증

* C.reception_yn<>'Y' OR B_latest.offer_limit=0 인데 실행 존재 ⇒ 비정상(0건이어야 함)

* final_loan_amount ≤ B_latest.offer_limit

* execution_date ≥ process_date


날짜/BQ 팁

* YYYY-mm-DD Date형

* 기준일: 신청=A.application_date, 조회=B_latest.inquiry_date, 접수=A.process_date, 실행=A.execution_date

* 기간 필터는 DATE 컬럼에 적용. 대용량은 DATE 파티션(예: application_date, inquiry_date) + CLUSTER BY(inquiry_id, loan_req_id) 권장.


예외/보정

* A.inquiry_id가 NULL/무효면, 동일 loan_req_id의 (offer_limit>0) 중 최저 금리 inquiry_id로 대체 허용.

* 재조회 다수일 때는 항상 B_latest 값만 사용.

#
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------

# 3. 분석